# Link features

So far, we haven't enabled ourselves to query for the features[^features] of ingested data, and used LaminDB like a data lake.

[^features]: We'll mostly use the term feature for synonyms variable (statistics), column and field (databases), dimension (machine learning).


We can also use LaminDB like a queryable data warehouse to store links[^relations] and monitor data integrity.

Let us explain how to implement this by providing feature models at ingestion!

[^relations]: We mostly use the term link for synonyms relations and references.

In [1]:
import lamindb as db
import bionty as bt  # https://lamin.ai/docs/bionty
import scanpy as sc  # https://scanpy.readthedocs.io

db.header()

id,ZKJX7AnXzQQp
version,draft
time_init,2022-07-30 16:14
time_run,2022-08-23 13:29
pypackage,bionty==0.1.0 lamindb==0.2.1 scanpy==1.9.1


## Example datasets

Consider
- `data1`: a flow cytometry dataset in form of an `.fcs` file
- `data2`: a scRNA-seq count matrix in form of an `AnnData` object in memory

In [2]:
data1 = db.datasets.file_fcs()
data1

PosixPath('example.fcs')

In [3]:
data2 = sc.read(db.datasets.file_mouse_sc_lymph_node())

## Define feature models

For `data1`, we specify a feature model using the `bionty` `Gene` entity with gene symbols.

In [4]:
feature_model1 = bt.Gene(id=bt.lookup.gene_id.name, species=bt.lookup.species.human)

Let us now ingest the data by passing a feature model to `db.do.ingest.add`, which will enable us to query the `dobject` by features by creating all necessary links in the background.

It will also log out and store information on data integrity:

In [5]:
db.do.ingest.add(data1, feature_model=feature_model1)

🔶 name column not found, using index as features.


✅ 9 terms (56.2%) are linked.


🔶 7 terms (43.8%) are not linked.


Using this feature model, we can't link and hence won't be able to query for 9 features.

We can overcome this by working with a custom feature model, discussed later.

Features in data2 are indexed by Ensemble gene ids. For an overview of gene ids, see: [`bt.lookup.gene_id`](https://lamin.ai/docs/bionty/api).

In [6]:
data2.var.head()

""
ENSMUSG00000104923
ENSMUSG00000079038
ENSMUSG00000069755
ENSMUSG00000040648
ENSMUSG00000113486


Hence, we use a feature model based on Ensemble IDs and ingest the data with it.

In [7]:
feature_model2 = bt.Gene(
    id=bt.lookup.gene_id.ensembl_gene_id, species=bt.lookup.species.mouse
)

In [8]:
db.do.ingest.add(data2, name="mouse_sc_lymph_node", feature_model=feature_model2)

🔶 ensembl_gene_id column not found, using index as features.


✅ 10000 terms (100.0%) are linked.


🔶 0 terms (0.0%) are not linked.


We can retrieve the integrity information through `.logs`:

In [9]:
db.do.ingest.logs

{'example.fcs': {'feature': 'name',
  'n_mapped': 9,
  'percent_mapped': 56.2,
  'unmapped': Index(['FSC-A', 'FSC-H', 'SSC-A', 'KI67', 'CD3', 'CD45RO', 'VIVID / CD14'], dtype='object')},
 'mouse_sc_lymph_node.h5ad': {'feature': 'ensembl_gene_id',
  'n_mapped': 10000,
  'percent_mapped': 100.0,
  'unmapped': Index([], dtype='object')}}

Finalize the ingestion.

In [10]:
db.do.ingest.commit()

✅ Annotated data n3Rz8JrhV5L25erSQeRFs with the following features:
+------------+------------+------------+
| geneset.id | biometa.id | species.id |
+------------+------------+------------+
|     3      |     3      |     1      |
+------------+------------+------------+


✅ Registered readout_type: 2


✅ Annotated data 5STsR8esNAlxTO9yaFtJN with the following features:
+------------+------------+------------+
| geneset.id | biometa.id | species.id |
+------------+------------+------------+
|     4      |     4      |     2      |
+------------+------------+------------+


✅ Ingested the following dobjects:
+-----------------------------------------------------+-----------------------------------+-----------------------+
|                       dobject                       |              jupynb               |         user          |
+-----------------------------------------------------+-----------------------------------+-----------------------+
|       example.fcs (n3Rz8JrhV5L25erSQeRFs, 1)        | 'Link features' (ZKJX7AnXzQQp, 1) | test-user2 (GWoum48D) |
| mouse_sc_lymph_node.h5ad (5STsR8esNAlxTO9yaFtJN, 1) | 'Link features' (ZKJX7AnXzQQp, 1) | test-user2 (GWoum48D) |
+-----------------------------------------------------+-----------------------------------+-----------------------+


✅ Cell numbers increase at increments of 1: Awesome!


ℹ️ Set notebook version to 1 & wrote pypackages.
